In [1]:
# !pip install -q --upgrade torch accelerate kernels
# !pip install -q git+https://github.com/huggingface/transformers triton==3.4 git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels
# !pip uninstall -q torchvision torchaudio -y

In [2]:
gpt = False

In [3]:
import sys
sys.path.append('../src')
import paths

/home/user/mnlp/notebooks/../src/paths.py


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, Mxfp4Config
if gpt:

    model_id = "openai/gpt-oss-20b"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    config = AutoConfig.from_pretrained(model_id)

    quantization_config=Mxfp4Config.from_dict(config.quantization_config)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        torch_dtype="auto",
        device_map="cuda",
    )

/home/user/mnlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import pandas as pd
import dataset
import importlib
importlib.reload(dataset)
import gptoss_sent_split
importlib.reload(gptoss_sent_split)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from gptoss_sent_split import BOSConfig, read_token_label_file, build_bos_jobs_by_n_sentences, run_bos_labeling, sentences_from_word_seq
if gpt:

    cfg = BOSConfig(max_new_tokens=256)

    pairs = read_token_label_file(paths.data/"manzoni_train_tokens.csv")
    jobs = build_bos_jobs_by_n_sentences(pairs, tokenizer, cfg)
    y_pred = run_bos_labeling(jobs, model, tokenizer, cfg)

    tokens = [t for (t, _) in pairs]
    gold = [y for (_, y) in pairs]

    # Align lengths, just in case
    n = min(len(tokens), len(y_pred))
    tokens, gold, y_pred = tokens[:n], gold[:n], y_pred[:n]
    sents = sentences_from_word_seq(tokens, y_pred)
    import pickle
    with open(paths.results/'gptpredtrain.pkl', 'wb') as f:
        pickle.dump(y_pred, f)
import pickle
with open(paths.results/'gptpredtrain.pkl', 'rb') as f:
    y_pred = pickle.load(f)

pairs = read_token_label_file(paths.data/"manzoni_train_tokens.csv")
gold = [y for (_, y) in pairs]
print(len(gold))
print(len(y_pred))
prec, rec, f1, _ = precision_recall_fscore_support(
    gold, y_pred, labels=[1], average="binary", zero_division=0
)
acc = accuracy_score(gold, y_pred)
{"precision": float(prec), "recall": float(rec), "f1": float(f1), "accuracy": float(acc)}

74764
74764


{'precision': 0.6630260199164792,
 'recall': 0.8434818144666939,
 'f1': 0.7424460431654676,
 'accuracy': 0.9808463966614949}

In [13]:
import pandas as pd
import dataset
import importlib
importlib.reload(dataset)
import gptoss_sent_split
importlib.reload(gptoss_sent_split)
from gptoss_sent_split import BOSConfig, read_token_label_file, build_bos_jobs_by_n_sentences, run_bos_labeling, sentences_from_word_seq
if gpt:

    cfg = BOSConfig(max_new_tokens=256)

    pairs = read_token_label_file(paths.data/"manzoni_dev_tokens.csv")
    jobs = build_bos_jobs_by_n_sentences(pairs, tokenizer, cfg)
    y_pred = run_bos_labeling(jobs, model, tokenizer, cfg)

    tokens = [t for (t, _) in pairs]
    gold = [y for (_, y) in pairs]

    # Align lengths, just in case
    n = min(len(tokens), len(y_pred))
    tokens, gold, y_pred = tokens[:n], gold[:n], y_pred[:n]
    sents = sentences_from_word_seq(tokens, y_pred)
    import pickle
    with open(paths.results/'gptpredval.pkl', 'wb') as f:
        pickle.dump(y_pred, f)
with open(paths.results/'gptpredval.pkl', 'rb') as f:
    y_pred = pickle.load(f)

pairs = read_token_label_file(paths.data/"manzoni_dev_tokens.csv")
gold = [y for (_, y) in pairs]
print(len(gold))
print(len(y_pred))
prec, rec, f1, _ = precision_recall_fscore_support(
    gold, y_pred, labels=[1], average="binary", zero_division=0
)
acc = accuracy_score(gold, y_pred)
{"precision": float(prec), "recall": float(rec), "f1": float(f1), "accuracy": float(acc)}

9343
9343


{'precision': 0.6625,
 'recall': 0.8179012345679012,
 'f1': 0.7320441988950276,
 'accuracy': 0.979235791501659}